# ViT Model Evaluation Notebook
This notebook loads your saved ViT model (`nn-vt-org-weights.pth`) and runs inference on test images.

In [1]:

import torch
import timm
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model EXACTLY like training
num_classes = 7
model = timm.create_model("vit_base_patch16_224", pretrained=True)
in_features = model.head.in_features
model.head = nn.Linear(in_features, num_classes)

# Load weights
model.load_state_dict(torch.load("nn-vt-org-weights.pth", map_location=device))
model.to(device)
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


In [2]:

# Image transforms (same as training)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [3]:

# Single image prediction function

def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, pred = torch.max(outputs, 1)

    print("Predicted class index:", pred.item())
    return pred.item()

# Example:
# predict_image("test.jpg")


In [4]:
import torch
import torch.nn as nn
import timm # Ensure you have this library installed: pip install timm
import os

# --- 1. CONFIGURATION (Must match your training setup) ---
model_name = "vit_base_patch16_224" # The exact model used in training
num_classes = 7                      # The number of classes you trained for
feature_extract = False              # Set to False if you trained the whole model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
WEIGHTS_PATH = 'nn-vt-org-weights.pth' # The path to your saved .pth file

print(f"Loading model on device: {device}")


# --- 2. MODEL ARCHITECTURE DEFINITION (Copied from your notebook) ---

def set_parameter_requires_grad(model, feature_extracting):
    """Sets whether parameters should be frozen (for feature extraction)."""
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    """Initializes the Vision Transformer (ViT) model."""
    
    # Initialize the model structure from timm
    model_ft = timm.create_model(model_name, pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    
    # Get the number of input features for the final layer
    num_ftrs = model_ft.head.in_features
    
    # Replace the final classification head to match your number of classes
    model_ft.head = nn.Linear(num_ftrs, num_classes)
    
    # The expected input size for this specific ViT model
    input_size = 224
    
    return model_ft, input_size


# --- 3. MODEL LOADING LOGIC ---

def load_trained_model():
    """Initializes the model structure and loads the saved state dictionary."""
    
    # 1. Initialize the model structure
    # use_pretrained=True is fine here, as the next step will overwrite the weights anyway.
    model_ft, input_size = initialize_model(
        model_name, num_classes, feature_extract, use_pretrained=True
    )
    model_ft = model_ft.to(device)

    # 2. Load the saved weights
    if not os.path.exists(WEIGHTS_PATH):
        print(f"❌ Error: Weights file not found at '{WEIGHTS_PATH}'")
        return None

    try:
        # Load the state dictionary from the saved file, mapping to the current device
        model_ft.load_state_dict(torch.load(WEIGHTS_PATH, map_location=device))
        
        # Set the model to evaluation mode (crucial for inference/testing)
        model_ft.eval()
        
        print(f"✅ Successfully loaded and set up the model from '{WEIGHTS_PATH}'.")
        return model_ft

    except Exception as e:
        print(f"❌ An error occurred while loading the state dictionary: {e}")
        return None

Loading model on device: cuda:0


In [8]:
loaded_model = load_trained_model()

if loaded_model:
        # You can now use 'loaded_model' for prediction or evaluation
    print("\nModel is ready for use.")
        # Example of making a prediction (placeholder for a single image tensor)
        # sample_input = torch.randn(1, 3, 224, 224).to(device) 
        # with torch.no_grad():
        #     output = loaded_model(sample_input)
        #     print(f"Sample prediction output shape: {output.shape}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.67 GiB of which 4.94 MiB is free. Process 4153 has 4.97 GiB memory in use. Including non-PyTorch memory, this process has 684.00 MiB memory in use. Of the allocated memory 541.39 MiB is allocated by PyTorch, and 60.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)